In [37]:
#######FIT THE MODEL ON HISTORICAL DATA!!!

import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse

filepath = '/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/models-links/'
train_df = pd.read_csv(filepath + 'radshift-links-previous-to-next-page-info.csv')
# train_df = build_df
train_df = train_df.fillna(0)
all_pages = train_df['page'].unique()

###encode page info - labels
from sklearn import preprocessing
curr_le = preprocessing.LabelEncoder()
curr_le = curr_le.fit(train_df['page'])
train_df['current_page_label'] = curr_le.transform(train_df['page'])
next_le = preprocessing.LabelEncoder()
next_le = next_le.fit(train_df['next_page'])
train_df['next_page_label'] = next_le.transform(train_df['next_page'])

one_hour_features = [
       'consumptions_by_type__link_clicks',
      'consumptions_by_type__other_clicks',
      'reactions_like_total',
      'stories_by_action_type__comment',
      'stories_by_action_type__like',
       'stories_by_action_type__share', 
        'engaged_fan',
      'fan_reach', 
    'impressions',
      'impressions_fan',
                ]

# cross_promotion_features = all_pages

from sklearn import linear_model
clf = linear_model.LinearRegression()
print("LINEAR REGRESSION!")

# from sklearn.tree import DecisionTreeRegressor
# clf = DecisionTreeRegressor(max_depth = 5)
# print("TREE REGRESSION!")

print("Performance stats with page names, and also cross-promotion order included!")
# print("WITH cross-promotion features")

page_infos = ['current_page_label', 'next_page_label']
# page_infos = ['page', 'next_page']
# page_infos = ['display_name', 'next_page']   ###mode report variables

train_features = one_hour_features

# include_page_info = 0
include_page_info = 1
if include_page_info == 1:
  train_features = one_hour_features + page_infos
# train_features = cross_promotion_features + one_hour_features

log_scale = 1
# log_scale = 0

if log_scale == 1:
    for feat in one_hour_features:
        train_df[feat] = np.log(train_df[feat] + 1)

cross_promote = 0
# cross_promote = 1

if cross_promote == 1:
    for feat in cross_promotion_features:
        train_features.append(feat)

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)
train_df['share_bucket'] = np.log(train_df['next_shares']+1)

print("No. of data-points = ", len(train_df))

###generate DFs for analysis - X and Y
X = train_df[train_features + ['title']]
y = train_df['share_bucket']
# y = train_df['next_shares']
####convert page_infos into dummies
# if include_page_info == 1:
#   X = pd.get_dummies(data = X, columns=['display_name', 'next_page']) ###mode report variables
#   X = pd.get_dummies(data = X, columns=['page', 'next_page']) 

####include title features!
title_vect = Pipeline([
                     ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ])

##
X_tfidf = title_vect.fit_transform(X['title'])
X = X.drop(['title'], axis = 1)
X = scipy.sparse.hstack([X_tfidf, X])

clf = clf.fit(X, y)
clf

LINEAR REGRESSION!
Performance stats with page names, and also cross-promotion order included!
No. of data-points =  38769


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [56]:
####make predictions on performance if posted on the target page, given the perfoprmance in the previous page!
test_page = 'BuzzFeed Animals'
max_suggestions = 10
test_df = train_df

test_df['test_page'] = test_page
test_df['next_page_label'] = next_le.transform(test_df['test_page'])

X_test = test_df[train_features + ['title']]

###save for later!
# curr_page = X_test['page']

# if include_page_info == 1:
# #   X_test = pd.get_dummies(data = X_test, columns=['display_name', 'next_page'])
#   X_test = pd.get_dummies(data = X_test, columns=['page', 'next_page'])
    
# for page in test_df['next_page'].unique():
#   X_test['next_page_' + page] = 0

# X_test['next_page_' + test_page] = 1

##title features
X_tfidf = title_vect.transform(X_test['title'])
X_test = X_test.drop(['title'], axis = 1)
X_test = scipy.sparse.hstack([X_tfidf, X_test])
###

test_df['predicted_shares'] = clf.predict(X_test)
test_df['predicted_shares'] = (np.exp(test_df['predicted_shares'])-1).astype(int)

result_cols = ['page', 'title', 'buzz_id', 'predicted_shares', 'next_page', 'test_page']
# result_cols = ['display_name', 'title', 'buzz_id', 'predicted_shares']
# test_df = test_df[~(test_df['page'] == test_page)]
# test_df = test_df[~(test_df['display_name'] == test_page)]
# ###only keep candidates that are from pages which have sourced test_page in the past!
top_k_similar = 10
candidates = test_df[test_df.next_page == test_page][['page', 'next_page']]
candidates['freq'] = candidates.groupby('page')['page'].transform('count')
candidates = candidates.drop_duplicates()
candidate_pages = candidates.sort_values('freq', ascending = False)['page'][:top_k_similar]
# candidate_pages = test_df[test_df.next_page == test_page]['page'].unique()
test_df = test_df[test_df['page'].isin(candidate_pages)]
###
test_df = test_df[result_cols].drop_duplicates(subset=['buzz_id'], keep = 'last')
test_df.sort_values('predicted_shares', ascending = False)[:max_suggestions]

,page,title,buzz_id,predicted_shares,next_page,test_page
26826,BuzzFeed News,Dakota Access Pipeline Will Be Rerouted In A V...,4412034.0,3037,BuzzFeed,BuzzFeed Animals
35383,BuzzFeed,Accidental Censorship Of Olympic Divers Makes ...,4330863.0,2946,BuzzFeed Video,BuzzFeed Animals
29878,BuzzFeed News,A Zoo Is Streaming Red Pandas On Tuesday To Di...,4392765.0,2859,BuzzFeed,BuzzFeed Animals
37519,BuzzFeed News,"Stop Trying To Rescue Baby Animals, Wildlife O...",4300956.0,2822,BuzzFeed,BuzzFeed Animals
3602,BuzzFeed News,These Are The Victims Of The Portland Train St...,4543535.0,2490,BuzzFeed,BuzzFeed Animals
29785,Cheeky,Thousands Of Americans Crashed Canada's Immigr...,4393851.0,2456,BuzzFeed Philippines,BuzzFeed Animals
38322,BuzzFeed,In Their Words: The Swedish Heroes Who Caught ...,4271370.0,2340,BuzzFeed Video,BuzzFeed Animals
33405,BuzzFeed,Ohio Police Share Graphic Photos Of Couple Ove...,4352719.0,1982,BuzzFeed Video,BuzzFeed Animals
29590,BuzzFeed News,"Middle Schoolers Chant ""Build The Wall"" In Caf...",4395258.0,1942,BuzzFeed Video,BuzzFeed Animals
37069,BuzzFeed News,You Should Probably Check Your Pokémon Go Priv...,4311070.0,1928,BuzzFeed Video,BuzzFeed Animals


In [ ]:
# print("Label encoding!!!")
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()

# le = le.fit(all_pages)
# le.classes_

In [53]:
# candidates = test_df[test_df.next_page == test_page][['page', 'next_page']]
# candidates['freq'] = candidates.groupby('page')['page'].transform('count')
# candidates = candidates.drop_duplicates()
# candidates.sort_values('freq', ascending = False)['page'][:10]

In [61]:
all_pages

array(['Buy Me That', 'BuzzFeed Style', 'BuzzFeed', 'BuzzFeed Video',
       'BuzzFeed Philippines', 'BuzzFeed Food', 'Tasty',
       'BuzzFeed Parents', 'BuzzFeed UK News', 'BuzzFeed Australia',
       'BuzzFeed IRL', 'BuzzFeed Animals', 'BuzzFeed UK',
       'Obsessed by BuzzFeed', 'SOML', 'BuzzFeed News',
       'BuzzFeed Entertainment', 'Quizzes En Español', 'BuzzFeed México',
       'BuzzFeed DIY', 'BuzzFeed Books', 'BuzzFeed Quiz', 'BuzzFeed World',
       'BuzzFeed Rewind', 'BuzzFeed LGBT', 'BuzzFeed Celeb',
       'BuzzFeed Geeky', 'BuzzFeed Ladylike', 'BuzzFeed Español',
       'BuzzFeed Science', 'Cheeky', 'BuzzFeed Health',
       'BuzzFeed Community', 'BuzzFeed Brasil', 'BuzzFeed India',
       'Cocoa Butter', 'BuzzFeed Politics', 'BuzzFeed Weddings',
       'BuzzFeed Japan News', 'LOLA', 'BuzzFeed Scotland',
       'BuzzFeed Oz Politics', 'BuzzFeed Canada', 'BuzzFeed Oz News',
       'BuzzFeed España', 'BuzzFeed UK Politics', 'Pero Like',
       'BuzzFeed France', 'Reasons